## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2024_10_10_random3try2.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [5]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [6]:
sample_table = pd.read_csv('SampleTable_2024_11_14_OptimizationRound3.csv')

In [7]:
stober.count_stock_vials(sample_table, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 2}

In [8]:
sample_table['ethanol_volume'].sum()/10000

2.783978108884767

## Load Tools

In [9]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [10]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [11]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [28]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [29]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [30]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [31]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [32]:
water_syringe.load_syringe(750, water_syringe.full_position+150)

Loaded syringe, remaining volume 750 uL


## Load ammonia syringe

In [33]:
ammonia_syringe.set_pulsewidth(1420, s = 10)

In [34]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [35]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [36]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [37]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+350, s = 500)

In [38]:
ammonia_syringe.load_syringe(370, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 370 uL


## Load TEOS syringe

In [39]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [40]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [41]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 100)

In [42]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [43]:
teos_syringe.load_syringe(690, teos_syringe.full_position+200)

Loaded syringe, remaining volume 690 uL


## load ethanol syringe

In [44]:
#jubilee.park_tool()

In [45]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [46]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [47]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 10)

In [48]:
mix_syringe.empty_position

1830

In [49]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+400, s = 500)

In [50]:
mix_syringe.load_syringe(3000, mix_syringe.full_position+400)

Loaded syringe, remaining volume 3000 uL


## Load labware


In [14]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(17.7,173.0), (130.7, 173.3), (131.1, 103.4)])

-0.004188629607457991
New manual offset applied to septavialrev1_44_holder_2000ul


In [15]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(30.9, 262.3), (114.9, 262.3), (114.2, 207.3)])

-0.006363292795721208
New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [16]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [17]:
teos_stocks = [stocks[0]]
ammonia_stocks = [stocks[1]]
water_stocks = [stocks[2]]
ethanol_stocks = [stocks[3], stocks[4], stocks[5], stocks[6], stocks[7], stocks[8]]
rinse_stocks = [stocks[9], stocks[10], stocks[11]]

## Check sample alignment

In [26]:
jubilee.pickup_tool(mix_syringe)
for well in stocks:
    jubilee.move_to(x = well.x, y = well.y)
    jubilee.move_to(z = well.top_+7)
    ans = input(f'Is the tip centered over well?')
    if ans == 'y':
        continue
    else:
        continue

Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 


In [24]:
jubilee.pickup_tool(teos_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    jubilee.move_to(z = loc.top_+2)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

Is the tip centered over sample A1? 
Is the tip centered over sample A2? 
Is the tip centered over sample A3? 
Is the tip centered over sample A4? 
Is the tip centered over sample A5? 
Is the tip centered over sample A6? 
Is the tip centered over sample A7? 
Is the tip centered over sample A8? 
Is the tip centered over sample B1? 
Is the tip centered over sample B3? 
Is the tip centered over sample B4? 
Is the tip centered over sample B5? 
Is the tip centered over sample B6? 
Is the tip centered over sample B8? 
Is the tip centered over sample C1? 
Is the tip centered over sample C2? 
Is the tip centered over sample C3? 
Is the tip centered over sample C4? 
Is the tip centered over sample C5? 
Is the tip centered over sample C6? 


In [27]:
jubilee.park_tool()

# Experiment


In [51]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}

In [52]:
dilution_table = sample_table.copy()

In [53]:
dilution_table['well'] = dilution_table['dilute_well']

In [54]:
location_lookup_dilute = {row['uuid']:samples[row['well']] for i, row in dilution_table.iterrows()}

## add ethanol

In [81]:
jubilee.park_tool()

In [24]:
mix_syringe.remaining_volume

3300

In [55]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

In [56]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [57]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [58]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [59]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table, location_lookup, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [60]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup, rinse_stocks, 5)

## Add TEOS and mix

In [61]:
teos_start_time = time.time()

In [62]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table, location_lookup, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks))

### Dilution

In [63]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table, location_lookup_dilute, 'ethanol_dilute_vol', ethanol_stocks[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup[uuid_val]
    dilute_well = location_lookup_dilute[uuid_val]
    vol = row['sample_dilute_vol']
    
    mix_syringe.aspirate(vol, sample_well.bottom(+5))
    time.sleep(10)
    mix_syringe.dispense(vol, dilute_well.bottom(+5))
    time.sleep(10)
    logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')

    for stock in rinse_stocks:
        mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    

In [64]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,97eb1065-a266-4bf0-8d3d-70c6c45e135d,17.000000,59.886661,177.900108,1445.213231,2.232143,2.691584,0.003296,0.0,1700.0,A1,C7
1,1,e7968010-0689-42a7-8797-2a3e9b7ae66c,128.723552,59.976938,253.625649,1257.673861,0.294790,20.380603,0.024399,1199.0,501.0,A2,C8
2,2,fd6bad8e-aedb-430a-a5bd-e667c317cbb5,17.000000,8.500000,243.605258,1430.894743,2.232143,2.691584,0.003277,0.0,1700.0,A3,D1
3,3,9d522d04-006c-4226-847e-27f987ea389d,217.168911,125.968067,120.627048,1236.235975,0.174732,34.384022,0.041389,1403.0,297.0,A4,D2
4,4,9f62f2c7-89e5-4122-8d6c-f876026d5975,72.110605,81.826747,74.426442,1471.636206,0.526225,11.417162,0.014097,805.0,895.0,A5,D3
5,5,46a44e80-dbc4-4e33-906e-dcfc99b242f2,248.195589,102.837770,211.959846,1137.006795,0.152889,39.296428,0.046609,1440.0,260.0,A6,D4
6,6,23754685-93a7-40a9-aefd-34d561daf1a9,149.006813,65.518953,22.988700,1462.485534,0.254662,23.592021,0.029162,1267.0,433.0,A7,D5
7,7,c8a04429-83dd-4779-a587-8e58a018413f,225.206037,8.500000,199.486357,1266.807606,0.168496,35.656527,0.042781,1414.0,286.0,A8,D6
8,8,326de561-5b2e-46d0-a192-b9bff6174040,17.000000,186.999999,102.630764,1393.369238,2.232143,2.691584,0.003300,0.0,1700.0,B1,D7
9,9,ee49095a-c041-4633-b695-6863ffcee3fa,81.911343,138.443228,146.033549,1333.611880,0.463262,12.968898,0.015754,912.0,788.0,B3,D8


In [65]:
jubilee.park_tool()

In [100]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [101]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1)

In [102]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1)